_Camille Avestruz (UChicago)_, _Alex Malz (NYU)_, \& Others

Our understanding of how folks currently use this code is based on the [Analytic Bias Estimate (ABE)](https://github.com/LSSTDESC/clmassmod/blob/master/notebooks/Analytic_bias_estimate.ipynb) and [Statistical Power Check (SPC)](https://github.com/LSSTDESC/clmassmod/blob/master/notebooks/Statistical%20Power%20Check.ipynb) demos.

This is a wishlist for how we'd like to use `clmassmod`.  We will refactor the code to make this notebook run!

There was a lot of other stuff here but we think it was more about simulating mock data, not the analysis pipeline we want to work on.  It can be found in commit a103463.

In [ ]:
import clmassmod as clmm

In [ ]:
massmod = clm.CLMassMod()

In [ ]:
bin_definitions = np.linspace(low_lim, hi_lim, number_of_bins + 1)# or whatever

In [ ]:
bin_assignments, pdfs, mles = massmod.individual_profile_fits(model_name, in_data, calc_params, bin_definitions)
    # parallelization option in calc_params
    # writes one file per halo

In [ ]:
massmod.write(directory_name)

In [ ]:
massmod.read(directory_name)

In [ ]:
what_is_this = massmod.ensemble_fit(bin_definitions, method_name)
    # does the sampling as method_name kw
    # opens every file in directory

In [ ]:
likelihoods, posteriors, mass_dist_bias_samps, mass_dist_scatter_samps = massmod.compare(truth)